In [ ]:
# Copyright 2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vision and Sensing Application SDK in a day

Using ["**Zone Detection**"](https://developer.aitrios.sony-semicon.com/development-guides/tutorials/sample-application/) sample application of tutorials in AITRIOS developer site as an example,

this jupyter notebook demonstrates an overview of the "**Vision and Sensing Application SDK**" in a day.

You can run this jupyter notebook step-by-step (one cell a time).

> **NOTE**
>
> TensorFlow and some library may print warning logs in output cell. Please ignore them.

## Imports

In [ ]:
import json
import subprocess
from pathlib import Path
from typing import Any, List, Tuple

import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

## Defines

### AI model dependent defines

In [ ]:
# You must not change following defines.

# for AI model trained in this notebook using TensorFlow Object Detection API
INPUT_SIZE = 300
DNN_OUTPUT_DETECTIONS = 10

### "**Vision and Sensing Application**" dependent defines

You can customize following **`MAX_DETECTIONS`** and **`APP_VERSION_NUMBER`**.

In [ ]:
# Vision and Sensing Application can use this parameter to limit the the number of objects
# detected by AI model in an image. In this case, if AI model detects more than 10 obejects,
# Vision and Sensing Application outputs only 10 objects which has higher precision.
MAX_DETECTIONS = 10

# version number of Vision and Sensing Application for Zone Detection
APP_VERSION_NUMBER = "01.01.00"

### This notebook dependent defines

In [ ]:
# You must not change following defines. You can skip reading them to understand the SDK.

SDK_ENV_ROOT_DIR = "../.."

VNSAPP_CODE_CLONE_DIR = (
    f"{SDK_ENV_ROOT_DIR}"
    "/.devcontainer/dependencies/aitrios-sdk-zone-detection-webapp-cs"
)
VNSAPP_CODE_DIR = "sample"
VNSAPP_DIR = "application"

## Common function

This cell defines a general utility function. So you can skip reading it to understand the SDK.

In [ ]:
def run_shell_command(command: str):
    """Run shell command with output log and checking return code."""

    with subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        shell=True,
        stderr=subprocess.STDOUT,
        bufsize=1,
        close_fds=True,
    ) as process:
        # for line in iter(process.stdout.readline, b""):
        while True:
            line = process.stdout.readline()
            if line:
                print(line.rstrip().decode("utf-8"))
            if process.poll() is not None:
                break
        process.stdout.close()
        return_code = process.wait()
        if return_code != 0:
            raise subprocess.CalledProcessError(process.returncode, process.args)

## Prepare dataset

Dataset for training AI model in this notebook is pre-installed in the SDK.

If you want to annotate image by yourself, please refer [../../tutorials/2_prepare_dataset/annotate_images/object_detection/README.md](../../tutorials/2_prepare_dataset/annotate_images/object_detection/README.md) for details.

### Defines

In [ ]:
# You must not change following defines. You can skip reading them to understand the SDK.

IMAGES_TRAINING_DIR = "./dataset/images/training"
IMAGES_VALIDATION_DIR = "./dataset/images/validation"
IMAGES_ORG_DIR = "./dataset/images_org"

### Extract images from dataset for training and validation.

In [ ]:
if not (Path(IMAGES_VALIDATION_DIR).is_dir() and Path(IMAGES_TRAINING_DIR).is_dir()):
    # Extract images
    run_shell_command(
        f"mkdir -p {IMAGES_ORG_DIR} \
            && unzip -o -j -d {IMAGES_ORG_DIR} \
            {VNSAPP_CODE_CLONE_DIR}/{VNSAPP_CODE_DIR}/training_images.zip"
    )

    # split training and validation images
    IMAGES_VALIDATION_FILES = "training_image_010.jpg training_image_020.jpg \
        training_image_025.jpg training_image_037.jpg training_image_045.jpg \
        training_image_052.jpg training_image_060.jpg"
    run_shell_command(
        f"mkdir -p {IMAGES_TRAINING_DIR} && mkdir -p {IMAGES_VALIDATION_DIR}"
    )
    run_shell_command(
        f"cd {IMAGES_ORG_DIR} \
            && mv {IMAGES_VALIDATION_FILES} ../../{IMAGES_VALIDATION_DIR}/ \
            && mv ./* ../../{IMAGES_TRAINING_DIR}/"
    )
    run_shell_command(f"rm -d {IMAGES_ORG_DIR}")
    print(f"Images extracted in {IMAGES_TRAINING_DIR} and {IMAGES_VALIDATION_DIR}")
else:
    print(
        f"Already extracted images in {IMAGES_TRAINING_DIR} and {IMAGES_VALIDATION_DIR}"
    )

### Load an image for evaluating (used as Input Tensor)

In [ ]:
IMAGE_INPUT = f"{IMAGES_VALIDATION_DIR}/training_image_037.jpg"
image = cv2.cvtColor(cv2.imread(IMAGE_INPUT), cv2.COLOR_BGR2RGB)
image = cv2.resize(image, dsize=(INPUT_SIZE, INPUT_SIZE))

## Prepare AI model

In this notebook, AI model is created and trained using TensorFlow Object Detection API.

### Defines

Training AI model takes several minutes to execute.

So if following **`is_always_retraining`** is False, training AI model executes only at first time.

If you want to train second time, please set **`is_always_retraining`** True.

In [ ]:
is_always_retraining = False

In [ ]:
# You must not change following defines. You can skip reading them to understand the SDK.

MODELS_DIR = "models"

TRAINING_DOCKER_IMAGE_NAME = "tf1_od_api_env:1.0.0"
TRAINING_DOCKER_VOLUME_DIR = "/root/samples/zone_detection"

### Set up environment for transfer learning (based on SSD MobileNet V1 AI model)

#### Download base AI model

In [ ]:
if not Path(f"./{MODELS_DIR}/out/ckpt").is_dir():
    BASE_AI_MODEL_FILE = "ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18"
    run_shell_command(
        f"wget \
            -q http://download.tensorflow.org/models/object_detection/{BASE_AI_MODEL_FILE}.tar.gz \
            -P ./{MODELS_DIR}/"
    )
    run_shell_command(f"mkdir -p ./{MODELS_DIR}/out")
    run_shell_command(
        f"tar -xvf ./{MODELS_DIR}/{BASE_AI_MODEL_FILE}.tar.gz -C ./{MODELS_DIR}/out"
    )
    run_shell_command(
        f"mv ./{MODELS_DIR}/out/{BASE_AI_MODEL_FILE} ./{MODELS_DIR}/out/ckpt"
    )
    run_shell_command(f"rm ./{MODELS_DIR}/{BASE_AI_MODEL_FILE}.tar.gz")
    print(f"Base AI model downloaded in ./{MODELS_DIR}/out/ckpt")
else:
    print(f"Already downloaded base AI model in ./{MODELS_DIR}/out/ckpt")

#### Build docker image for learning AI model using TensorFlow Object Detection API

> **NOTE**
>
> If the docker image isn't cached, it takes about 5 minutes.

In [ ]:
run_shell_command(
    f"docker build \
        --build-arg UBUNTU_VERSION=18.04 \
        --build-arg PIP=pip3 \
        --build-arg PYTHON=python3 \
        --build-arg USE_PYTHON_3_NOT_2=1 \
        --build-arg _PY_SUFFIX=3 \
        --build-arg TF_PACKAGE=tensorflow \
        --build-arg TF_PACKAGE_VERSION=1.15.5 \
        . -f Dockerfile \
        -t {TRAINING_DOCKER_IMAGE_NAME} --network host"
)

### Check accuracy of base model

#### Common functions

These cells define general utility function. So you can skip reading them to understand the SDK.

In [ ]:
def convert_to_tflite(output_file: str, graph_def_file: str):
    """Convert SavedModel to TFLite within training docker container."""

    output_arrays = (
        "TFLite_Detection_PostProcess,TFLite_Detection_PostProcess:1,"
        "TFLite_Detection_PostProcess:2,TFLite_Detection_PostProcess:3"
    )

    run_shell_command(
        f'docker run --rm -t -v $(pwd):{TRAINING_DOCKER_VOLUME_DIR} {TRAINING_DOCKER_IMAGE_NAME} \
            tflite_convert \
        --output_file={output_file} \
        --graph_def_file={graph_def_file} \
        --inference_type=QUANTIZED_UINT8 \
        --input_arrays="normalized_input_image_tensor" \
        --output_arrays={output_arrays} \
        --mean_values=128 \
        --std_dev_values=128 \
        --input_shapes=1,{INPUT_SIZE},{INPUT_SIZE},3 \
        --change_concat_input_ranges=false \
        --allow_nudging_weights_to_use_fast_gemm_kernel=true \
        --allow_custom_ops'
    )

In [ ]:
class TFLiteInterpreter:
    """A class to evaluate TFLite."""

    def __init__(self, model_file: str):
        self.interpreter = TFLiteInterpreter._make_interpreter(model_file)
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

    @staticmethod
    def _make_interpreter(model_file: str) -> Any:
        model_file, *device = model_file.split("@")
        interpreter = tf.lite.Interpreter(model_path=model_file)
        interpreter.allocate_tensors()
        return interpreter

    def _prepare_input(self, input_batch: np.ndarray) -> np.ndarray:
        if self.input_details[0]["dtype"] in [np.uint8, np.int8]:
            input_scale, input_zero_point = self.input_details[0]["quantization"]
            input_batch = input_batch / input_scale + input_zero_point
        return tf.cast(
            np.expand_dims(input_batch, axis=0), self.input_details[0]["dtype"]
        )

    @staticmethod
    def _get_output(interpreter, output_details, index):
        output = interpreter.get_tensor(output_details[index]["index"])
        if output_details[index]["dtype"] in [np.uint8, np.int8]:
            scale, zero_point = output_details[index]["quantization"]
            output = scale * (output - zero_point)
        return tf.cast(output, output_details[index]["dtype"])

    def _get_outputs(self) -> List[np.ndarray]:
        return [
            self._get_output(self.interpreter, self.output_details, index)
            for index in range(len(self.output_details))
        ]

    def run(self, input_batch: Any) -> None:
        self.interpreter.set_tensor(
            self.input_details[0]["index"], self._prepare_input(input_batch)
        )
        self.interpreter.invoke()
        return self._get_outputs()

In [ ]:
def evaluate_and_display_tflite(
    model_path: str, image: cv2.Mat, score_threshold: float, dnn_output_bboxes: int
) -> Tuple[list, list, cv2.Mat]:
    """Evaluate TFLite, convert Output Tensor format from TFLite to Edge AI Device and display
    the image overlaid with Output Tensor."""

    interp = TFLiteInterpreter(model_path)

    raw_output_tensors = interp.run(image.astype(np.float32) / 255.0)
    output_tensor = np.concatenate([np.array(x).flatten() for x in raw_output_tensors])

    plt.figure()

    num_of_detection = int(output_tensor[len(output_tensor) - 1])

    postprocessed = []
    for index in range(num_of_detection):
        inference = []
        inference.append(output_tensor[4 * dnn_output_bboxes + index])  # cls
        inference.append(output_tensor[5 * dnn_output_bboxes + index])  # score
        inference.append(output_tensor[0 + 4 * index])  # ymin
        inference.append(output_tensor[1 + 4 * index])  # xmin
        inference.append(output_tensor[2 + 4 * index])  # ymax
        inference.append(output_tensor[3 + 4 * index])  # xmax
        postprocessed.append(inference)

    # float array of Output Tensor on Edge AI Device
    output_tensor_device = [0.0 for _ in range(dnn_output_bboxes * 6 + 1)]
    output_tensor_device[dnn_output_bboxes * 6] = len(postprocessed)  # num of detection

    overlaid_text_image = np.zeros((INPUT_SIZE, INPUT_SIZE, 4), dtype=np.uint8)
    result_image = image.copy()
    result_image = cv2.cvtColor(result_image, cv2.COLOR_RGB2RGBA)
    for idx, x in enumerate(postprocessed):
        cls, score, ymin, xmin, ymax, xmax = x
        print(f"score: {score}, class id: {int(cls)}")

        output_tensor_device[dnn_output_bboxes * 0 + idx] = float(ymin)
        output_tensor_device[dnn_output_bboxes * 1 + idx] = float(xmin)
        output_tensor_device[dnn_output_bboxes * 2 + idx] = float(ymax)
        output_tensor_device[dnn_output_bboxes * 3 + idx] = float(xmax)
        output_tensor_device[dnn_output_bboxes * 4 + idx] = float(cls)
        output_tensor_device[dnn_output_bboxes * 5 + idx] = float(score)

        ymin = int(ymin * INPUT_SIZE)
        xmin = int(xmin * INPUT_SIZE)
        ymax = int(ymax * INPUT_SIZE)
        xmax = int(xmax * INPUT_SIZE)
        if score > score_threshold:
            cv2.rectangle(
                result_image, (xmin, ymin), (xmax, ymax), (255, 255, 0, 255), 2
            )
            text_pos_x = xmin + 2
            text_pos_y = ymax - 2
            if text_pos_x > (INPUT_SIZE - 35):
                text_pos_x = INPUT_SIZE - 35
            elif text_pos_x < 2:
                text_pos_x = 2
            if text_pos_y < 14:
                text_pos_y = 14
            elif text_pos_y > (INPUT_SIZE - 2):
                text_pos_y = INPUT_SIZE - 2
            cv2.putText(
                overlaid_text_image,
                str("{0:.0f}%".format(score * 100.0)),
                (text_pos_x, text_pos_y),
                cv2.FONT_HERSHEY_PLAIN,
                1,
                (255, 255, 0, 255),
                1,
                cv2.LINE_AA,
            )

    cv2.add(result_image, overlaid_text_image, result_image)
    plt.imshow(result_image)
    plt.axis("off")
    plt.show()
    plt.clf()
    plt.close()

    return output_tensor_device, postprocessed, result_image

#### Convert base AI model from SavedModel to TFLite

In [ ]:
if not Path(f"{MODELS_DIR}/base_model_quantized_od.tflite").is_file():
    OUTPUT_FILE = (
        f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/base_model_quantized_od.tflite"
    )
    GRAPH_DEF_FILE = (
        f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/out/ckpt/tflite_graph.pb"
    )
    convert_to_tflite(OUTPUT_FILE, GRAPH_DEF_FILE)
    print(
        f"Base AI model TFLite converted in ./{MODELS_DIR}/base_model_quantized_od.tflite"
    )
else:
    print(
        f"Already converted base AI model TFLite in ./{MODELS_DIR}/base_model_quantized_od.tflite"
    )

#### Evaluate and display TFLite base AI model

Since the AI model has not undergone transfer learning, the accurary will be low and cars will not be detected.

In [ ]:
MODEL = f"./{MODELS_DIR}/base_model_quantized_od.tflite"
SCORE_THRESHOLD = 0.3
output_tensor_device, postprocessed, result_image = evaluate_and_display_tflite(
    MODEL, image, SCORE_THRESHOLD, DNN_OUTPUT_DETECTIONS
)

### Transfer Learning

Using TensorFlow Object Detection API, Quantization Aware Training (QAT) is executed.

So the transfer learned AI model is already quantized as is.

#### Execute transfer learning

For customizing transfer learning parameters, you can change **`NUM_TRAIN_STEPS`** in following cell and **`batch_size`** in [./models/pipeline.config](./models/pipeline.config).

> **NOTE**
>
> It takes about 8 minutes.

> **NOTE**
>
> The training and validation TFRecord dataset is located in [./dataset/training](./dataset/training) and [./dataset/validation](./dataset/validation) and used in [./models/pipeline.config](./models/pipeline.config).
>
> The TFRecord dataset is created by CVAT annotation using [./dataset/images/training](./dataset/images/training) and [./dataset/images/validation](./dataset/images/validation) images.
>
> The annotated backuped CVAT project file is located in [./dataset/cvat_backup/](./dataset/cvat_backup).

> **NOTE**
>
> If you want to train again with more training steps using the same dataset as the previous training, please increase **`NUM_TRAIN_STEPS`** from the last time you trained.

In [ ]:
NUM_TRAIN_STEPS = 300

if (not Path(f"{MODELS_DIR}/out/train").is_dir()) or is_always_retraining:
    PIPELINE_CONFIG_PATH = f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/pipeline.config"
    MODEL_DIR = f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/out/train"
    SAMPLE_1_OF_N_EVAL_EXAMPLES = 100
    run_shell_command(
        f"docker run --rm -t -v $(pwd):{TRAINING_DOCKER_VOLUME_DIR} \
            --network host {TRAINING_DOCKER_IMAGE_NAME} \
            python /tensorflow/models/research/object_detection/model_main.py \
            --pipeline_config_path={PIPELINE_CONFIG_PATH} \
            --model_dir={MODEL_DIR} \
            --num_train_steps={NUM_TRAIN_STEPS} \
            --sample_1_of_n_eval_examples={SAMPLE_1_OF_N_EVAL_EXAMPLES} \
            --alsologtostderr"
    )
    print(f"Learned in ./{MODELS_DIR}/out/train")
else:
    print(f"Already learned in ./{MODELS_DIR}/out/train")

#### Export AI model as SavedModel

In [ ]:
if not Path(f"{MODELS_DIR}/out/models").is_dir() or is_always_retraining:
    TRAINED_CHECKPOINT_PREFIX = (
        f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/out/train/model.ckpt-{NUM_TRAIN_STEPS}"
    )
    OUTPUT_DIRECTORY = f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/out/models"
    run_shell_command(
        f"docker run --rm -t -v $(pwd):{TRAINING_DOCKER_VOLUME_DIR} {TRAINING_DOCKER_IMAGE_NAME} \
            python /tensorflow/models/research/object_detection/export_tflite_ssd_graph.py \
            --pipeline_config_path={PIPELINE_CONFIG_PATH} \
            --trained_checkpoint_prefix={TRAINED_CHECKPOINT_PREFIX} \
            --output_directory={OUTPUT_DIRECTORY} \
            --add_postprocessing_op=true"
    )
    print(f"Saved in ./{MODELS_DIR}/out/models")
else:
    print(f"Already saved in ./{MODELS_DIR}/out/models")

#### Convert transfer learned SavedModel to TFLite

In [ ]:
if (
    not Path(f"{MODELS_DIR}/model_quantized_od.tflite").is_file()
    or is_always_retraining
):
    OUTPUT_FILE = f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/model_quantized_od.tflite"
    GRAPH_DEF_FILE = (
        f"{TRAINING_DOCKER_VOLUME_DIR}/{MODELS_DIR}/out/models/tflite_graph.pb"
    )
    convert_to_tflite(OUTPUT_FILE, GRAPH_DEF_FILE)
    print(f"Converted in ./{MODELS_DIR}/model_quantized_od.tflite")
else:
    print(f"Already converted in ./{MODELS_DIR}/model_quantized_od.tflite")

#### Evaluate and display TFLite transfer learned AI model

Since the AI model has already undergone transfer learning, the accuracy will be high.

In [ ]:
MODEL = f"./{MODELS_DIR}/model_quantized_od.tflite"
SCORE_THRESHOLD = 0.0
output_tensor_device, postprocessed, result_image = evaluate_and_display_tflite(
    MODEL, image, SCORE_THRESHOLD, DNN_OUTPUT_DETECTIONS
)

#### Save Output Tensor to json file

The Output Tensor file will be used as input of "**Vision and Sensing Application**" in the SDK environment.

In [ ]:
with open(f"./{VNSAPP_DIR}/output_tensor.jsonc", "w") as fp:
    json.dump(output_tensor_device, fp, indent=4)
    print(f"Output Tensor saved as ./{VNSAPP_DIR}/output_tensor.jsonc")

## Prepare "**Vision and Sensing Application**"

### What is "**Vision and Sensing Application**"?

"**Vision and Sensing Application**" is post-processing application of AI model's Output Tensor running on Edge AI Device.

Please see [../../tutorials/4_prepare_application/1_develop/README.md](../../tutorials/4_prepare_application/1_develop/README.md) for details.

In "**Vision and Sensing Application**" for "**Zone Detection**", by calculating IoU (Intersection over Union) in Edge AI Device, there is an advantage that the amount of uploading data to the cloud can be reduced.

### Set up environment for developing "**Vision and Sensing Application**"

#### Copy Makefile customized for the SDK environment

In [ ]:
run_shell_command(
    f"cp {VNSAPP_DIR}/Makefile_VnS \
        {VNSAPP_CODE_CLONE_DIR}/{VNSAPP_CODE_DIR}/sample/vision_app/single_dnn/zonedetection/"
)

### Develop and build "**Vision and Sensing Application**"

#### Common function

In [ ]:
def build_wasm():
    """build Vision and Sensing Application and copy the compiled Wasm file to this notebook's
    directory"""

    run_shell_command(f"{VNSAPP_DIR}/build.sh")
    run_shell_command(
        f"cp -f {VNSAPP_CODE_CLONE_DIR}/{VNSAPP_CODE_DIR}"
        "/sample/vision_app/single_dnn/zonedetection/build/release/"
        "vision_app_zonedetection.wasm"
        f" ./{VNSAPP_DIR}/"
    )
    print(f"\nCompiled Wasm file path: ./{VNSAPP_DIR}/vision_app_zonedetection.wasm")

#### Modify the source code (if needed)

The source code exists in [./../../.devcontainer/dependencies/aitrios-sdk-zone-detection-webapp-cs/sample](./../../.devcontainer/dependencies/aitrios-sdk-zone-detection-webapp-cs/sample).

Please modify the source code if needed.

#### Build Wasm

"**Vision and Sensing Application**" is compiled to Wasm file.

> **NOTE**
>
> If docker image for building Wasm doesn't cached, it takes about 4 minutes.

In [ ]:
build_wasm()

## Run "**Vision and Sensing Application**" with mock

The SDK provides the test application for running "**Vision and Sensing Application**", that mocks dependency libraries of Edge AI Device firmware.

So you can run "**Vision and Sensing Application**" in the SDK environment for development and test.

Please see [../../tutorials/4_prepare_application/1_develop/README_wasmdebug.md](../../tutorials/4_prepare_application/1_develop/README_wasmdebug.md) for details.

### Defines

In [ ]:
# You must not change following defines. You can skip reading them to understand the SDK.

PPL_ENV_DIR = f"{SDK_ENV_ROOT_DIR}/tutorials/4_prepare_application/1_develop"
PPL_ENV_OUTPUT_DIR = "testapp/build/loader"
PPL_ENV_OUTPUT_FILE = "serialized_result_01.bin"

### Common functions

These cells define general utility function. So you can skip reading them to understand the SDK.

In [ ]:
def display_deserialized_inference_result(
    file_path: str, image: cv2.Mat, ppl_parameter
):
    """Display an image with overlaid metadata of inference result"""

    plt.figure()

    with open(file_path, "r") as f:
        json_load = json.load(f)

    detection_list = json_load["perception"]["object_detection_list"]

    overlaid_text_image = np.zeros((INPUT_SIZE, INPUT_SIZE, 4), dtype=np.uint8)
    result_image = image.copy()
    result_image = cv2.cvtColor(result_image, cv2.COLOR_RGB2RGBA)
    if (
        "zone" in ppl_parameter
        and "top_left_x" in ppl_parameter["zone"]
        and "top_left_y" in ppl_parameter["zone"]
        and "bottom_right_x" in ppl_parameter["zone"]
        and "bottom_right_y" in ppl_parameter["zone"]
    ):
        zone_xmin = ppl_parameter["zone"]["top_left_x"]
        zone_ymin = ppl_parameter["zone"]["top_left_y"]
        zone_xmax = ppl_parameter["zone"]["bottom_right_x"]
        zone_ymax = ppl_parameter["zone"]["bottom_right_y"]
        cv2.rectangle(
            result_image,
            (zone_xmin, zone_ymin),
            (zone_xmax, zone_ymax),
            (255, 0, 0, 255),
            2,
        )

    for idx, detection in enumerate(detection_list):
        bbox = detection["bounding_box"]
        score = detection["score"]
        iou = detection["iou"]
        zoneflag = detection["zoneflag"]
        xmin, ymin, xmax, ymax = bbox.values()
        if zoneflag:
            rectangle_color = (255, 255, 0, 255)
        else:
            rectangle_color = (255, 255, 0, 255)
        cv2.rectangle(result_image, (xmin, ymin), (xmax, ymax), rectangle_color, 2)

        text_pos_x = xmin + 2
        text_pos_y = ymin + 13
        if text_pos_x > (INPUT_SIZE - 35):
            text_pos_x = INPUT_SIZE - 35
        elif text_pos_x < 2:
            text_pos_x = 2
        if text_pos_y < 13:
            text_pos_y = 13
        elif text_pos_y > (INPUT_SIZE - 15):
            text_pos_y = INPUT_SIZE - 15
        cv2.putText(
            overlaid_text_image,
            str("{0:.0f}%".format(iou * 100.0)),
            (text_pos_x, text_pos_y),
            cv2.FONT_HERSHEY_PLAIN,
            1,
            (0, 255, 0, 255),
            1,
            cv2.LINE_AA,
        )

        text_pos_x = xmin + 2
        text_pos_y = ymax - 2
        if text_pos_x > (INPUT_SIZE - 35):
            text_pos_x = INPUT_SIZE - 35
        elif text_pos_x < 2:
            text_pos_x = 2
        if text_pos_y < 26:
            text_pos_y = 26
        elif text_pos_y > (INPUT_SIZE - 2):
            text_pos_y = INPUT_SIZE - 2
        cv2.putText(
            overlaid_text_image,
            str("{0:.0f}%".format(score * 100.0)),
            (text_pos_x, text_pos_y),
            cv2.FONT_HERSHEY_PLAIN,
            1,
            (255, 255, 0, 255),
            1,
            cv2.LINE_AA,
        )

    cv2.add(result_image, overlaid_text_image, result_image)
    plt.imshow(result_image)
    plt.axis("off")
    plt.show()
    plt.clf()
    plt.close()

In [ ]:
def deserialize_output_tensor(serialized_output_tensor_file_name: str):
    """Deserialize Output Tensor binary file to json file"""

    run_shell_command(
        f"cp $(pwd)/{VNSAPP_CODE_CLONE_DIR}/{VNSAPP_CODE_DIR}/schema/zonedetection.fbs \
            $(pwd)/deserialize/zonedetection.fbs"
    )
    run_shell_command(
        f"cd $(pwd)/deserialize/ \
        && ./binary_to_json.sh zonedetection.fbs {serialized_output_tensor_file_name} ./"
    )

### Prepare PPL Parameter and run "**Vision and Sensing Application**"

PPL Parameter is used for customizing behavior of "**Vision and Sensing Application**" by setting parameters.

#### Create PPL Parameter and save it as json

At first, use following PPL Parameter as provisional values for building and running "**Vision and Sensing Application**" in the SDK environment. The **`zone`** value is incorrect and the **`threshold`** value is set too high.

In [ ]:
ppl_parameter_before = {
    "header": {
        "id": "00",
        "version": APP_VERSION_NUMBER
    },
    "dnn_output_detections": DNN_OUTPUT_DETECTIONS,
    "max_detections": MAX_DETECTIONS,
    "mode": 0,  # mode of "Zone Detection" application. 0: detections not filtered by IoU, 1: detections filtered by IoU
    "zone": {
        "top_left_x": 10,
        "top_left_y": 100,
        "bottom_right_x": 200,
        "bottom_right_y": 250
    },
    "threshold": {
        "iou": 0.9,
        "score": 0.8
    },
    "input_width": INPUT_SIZE,
    "input_height": INPUT_SIZE
}

with open(f"./{VNSAPP_DIR}/ppl_parameter_before.json", "w") as fp:
    json.dump(ppl_parameter_before, fp, indent=4)
    print(f"PPL Parameter saved as ./{VNSAPP_DIR}/ppl_parameter_before.json")

#### Build and run "**Vision and Sensing Application**" in the SDK with mock library

Running "**Vision and Sensing Application**" in the SDK with mock library, metadata of inference result serialized by FlatBuffers is generated as a result.

To read the result, deserialize the metadata of inference result.

> **NOTE**
>
> If docker image for deserialize isn't cached, it takes about 3 minutes.

In [ ]:
build_wasm()

# Run Wasm on the SDK environment
run_shell_command(
    f"{VNSAPP_DIR}/start.sh \
                  -o $(pwd)/{VNSAPP_DIR}/output_tensor.jsonc \
                  -p $(pwd)/{VNSAPP_DIR}/ppl_parameter_before.json"
)

# Copy metadata of inference result from test application directory to this notebook directory
run_shell_command(
    f"mkdir -p $(pwd)/deserialize && \
        cp -f $(pwd)/{PPL_ENV_DIR}/{PPL_ENV_OUTPUT_DIR}/{PPL_ENV_OUTPUT_FILE} \
            $(pwd)/deserialize/ppl_output_before.bin"
)
print(
    "\nSerialized metadata of inference result binary file path: \
        ./deserialize/ppl_output_before.bin"
)

# Deserialize
BINARY_FILE = "ppl_output_before.bin"
deserialize_output_tensor(BINARY_FILE)
print(
    "\nDeserialized metadata of inference result json file path: \
        ./deserialize/ppl_output_before.json"
)

#### Display deserialized metadata of inference result

Since the PPL Parameter are provisional values, the result will not be expected.

In [ ]:
file_path = "./deserialize/ppl_output_before.json"
display_deserialized_inference_result(file_path, image, ppl_parameter_before)

# Display legend
# Yellow rectangle: detected object
# Yellow text: inference score of detected object
# Red rectangle: zone defined in PPL Parameter
# Green text: IoU (Intersection over Union) with detected object and zone

### Modify PPL Parameter and run "**Vision and Sensing Application**"

#### Create PPL Parameter and save it as json (second time)

Before running following cell, you can customize the PPL Parameter.

For example, following **`mode`**, **`zone`** and **`threshold`**.

In [ ]:
ppl_parameter_after = {
    "header": {
        "id": "00",
        "version": "01.01.00"
    },
    "dnn_output_detections": DNN_OUTPUT_DETECTIONS,
    "max_detections": MAX_DETECTIONS,
    "mode": 0,  # mode of "Zone Detection" application. 0: detections not filtered by IoU, 1: detections filtered by IoU
    "zone": {
        "top_left_x": 10,
        "top_left_y": 168,
        "bottom_right_x": 198,
        "bottom_right_y": 258
    },
    "threshold": {
        "iou": 0.5,
        "score": 0.4
    },
    "input_width": INPUT_SIZE,
    "input_height": INPUT_SIZE
}

with open(f"./{VNSAPP_DIR}/ppl_parameter_after.json", "w") as fp:
    json.dump(ppl_parameter_after, fp, indent=4)
    print(f"PPL Parameter saved as ./{VNSAPP_DIR}/ppl_parameter_after.json")

#### Build and run "**Vision and Sensing Application**" in the SDK with mock library (second time)

In [ ]:
build_wasm()

# Run Wasm on the SDK environment
run_shell_command(
    f"{VNSAPP_DIR}/start.sh \
        -o $(pwd)/{VNSAPP_DIR}/output_tensor.jsonc \
        -p $(pwd)/{VNSAPP_DIR}/ppl_parameter_after.json"
)

# Copy metadata of inference result from test application directory to this notebook directory
run_shell_command(
    f"mkdir -p $(pwd)/deserialize && \
        cp -f $(pwd)/{PPL_ENV_DIR}/{PPL_ENV_OUTPUT_DIR}/{PPL_ENV_OUTPUT_FILE} \
            $(pwd)/deserialize/ppl_output_after.bin"
)
print(
    "\nSerialized metadata of inference result binary file path: \
        ./deserialize/ppl_output_after.bin"
)

# Deserialize
BINARY_FILE = "ppl_output_after.bin"
deserialize_output_tensor(BINARY_FILE)
print(
    "\nDeserialized metadata of inference result json file path: \
        ./deserialize/ppl_output_after.json"
)

#### Display deserialized metadata of inference result (second time)

Since the PPL Parameter are configured values, the result will be expected.

In [ ]:
file_path = "./deserialize/ppl_output_after.json"
display_deserialized_inference_result(file_path, image, ppl_parameter_after)

# Display legend
# Yellow rectangle: detected object
# Yellow text: inference score of detected object
# Red rectangle: zone defined in PPL Parameter
# Green text: IoU (Intersection over Union) with detected object and zone

## Deploy AI model and "**Vision and Sensing Application**"

### Upload AI model to Azure Blob Storage and generate SAS URL

To deploy AI model to your Edge AI Device using this notebook, you have to prepare Azure Blob Storage by yourself.

Please upload the AI model [./models/model_quantized_od.tflite](./models/model_quantized_od.tflite) to Azure Blob Storage and generate SAS URL of the uploaded AI model using Azure Blob Storage.

About SAS URL, please see ["**Console User Manual**"](https://developer.aitrios.sony-semicon.com/en/documents/console-user-manual) for details.

To import AI model from local environment, please see ["**Console User Manual**"](https://developer.aitrios.sony-semicon.com/en/documents/console-user-manual).

### Imports

In [ ]:
import base64
import copy
import errno
import os

import jsonschema
import pandas as pd
from console_access_library.client import Client
from console_access_library.common.config import Config
from IPython.display import display

### Defines

In [ ]:
# You must not change following defines. You can skip reading them to understand the SDK.

CONNECTION_CONFIG_PATH = (
    f"{SDK_ENV_ROOT_DIR}/tutorials/_common/set_up_console_client/configuration.json"
)
CONNECTION_CONFIG_SCHEMA_PATH = (
    f"{SDK_ENV_ROOT_DIR}"
    "/tutorials/_common/set_up_console_client/configuration_schema.json"
)

### Common function

This cell defines a general utility function. So you can skip reading it to understand the SDK.

In [ ]:
def validate_symlink(path: Path):
    """Validate symbolic link"""

    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )

### Prepare configurations for connection

#### Create Configuration file for connection (only at first time)

In [ ]:
configuration_path = Path(f"{CONNECTION_CONFIG_PATH}")
if not configuration_path.is_file():
    with open(configuration_path, "w") as fp:
        configuration_connection = {
            "console_endpoint": "",
            "portal_authorization_endpoint": "",
            "client_secret": "",
            "client_id": ""
        }
        json.dump(configuration_connection, fp, indent=4)
    print(f"./{CONNECTION_CONFIG_PATH} is created.")
else:
    print(f"./{CONNECTION_CONFIG_PATH} already exists.")

#### Edit Configuration file for connection

Edit the parameters in [./../../tutorials/_common/set_up_console_client/configuration.json](../../tutorials/_common/set_up_console_client/configuration.json).

The parameters required to run this notebook are :

|Setting|Range|Required/Optional|Remarks
|:--|:--|:--|:--|
|**`console_endpoint`**|String.<br>See NOTE.|Required|Used for Console Access Library API: **`common.config.Config`**
|**`portal_authorization_endpoint`**|String.<br>See NOTE.|Required|Used for Console Access Library API: **`common.config.Config`**
|**`client_secret`**|String.<br>See NOTE.|Required|Used for Console Access Library API: **`common.config.Config`**
|**`client_id`**|String.<br>See NOTE.|Required|Used for Console Access Library API: **`common.config.Config`**

> **NOTE**
>
> See [API Reference](https://developer.aitrios.sony-semicon.com/development-guides/reference/api-references) of Console Access Library for other restrictions.

### Load Configurations for connection

In [ ]:
configuration_path = Path(f"{CONNECTION_CONFIG_PATH}")
validate_symlink(configuration_path)

# Load configuration file.
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path(f"{CONNECTION_CONFIG_SCHEMA_PATH}")
validate_symlink(configuration_schema_path)

# Load configuration schema file.
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration.
jsonschema.validate(json_load, json_schema)

print(f"./{CONNECTION_CONFIG_PATH} is loaded.")

### Set up Console Access Library

#### Instantiate Console Access Library and get device list with AI model info

In [ ]:
def get_devices_and_ai_models(client_obj: Client):
    """Get device list with AI model info"""

    # Get an instance of device management API
    device_management_obj = client_obj.get_device_management()

    # Call an API for get device list
    try:
        response = device_management_obj.get_devices()
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    print("Get device list complete.")

    # Create output list
    model_lists = []
    devices = response.get("devices", [])

    for device in devices:
        device_id = device.get("device_id", "")

        model_list = []
        model_list.append(device_id)
        models = device.get("models", [])
        if any(models):
            for model in models:
                version_id = ""
                model_version_id = model.get("model_version_id", "")
                if len(model_version_id) == 0:
                    # There is no models deployed on the device
                    model_id = ""
                elif ":" not in model_version_id:
                    # The model deployed to the device does not exist in Console database
                    model_id = model_version_id
                else:
                    # Split string to model_id and version_id
                    model_version_id_list = model_version_id.split(":v")

                    model_id = model_version_id_list[0]
                    version_id = model_version_id_list[1]

                model_list.append(model_id)
                model_list.append(version_id)

            # Fill missing elements with empty string
            model_list = model_list + ["" for x in range(9 - len(model_list))]

            model_lists.append(model_list)
        else:
            # There is no models deployed on the device
            model_list.append("")

            # Fill missing elements with empty string
            model_list = model_list + ["" for x in range(9 - len(model_list))]

            model_lists.append(model_list)

    if len(model_lists) == 0:
        raise Exception("There is no data in the device list.")

    output_frame = pd.DataFrame(
        model_lists,
        columns=["Device ID", "c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8"],
    )

    # Specify "Device ID" column as index row
    output_frame.set_index("Device ID", inplace=True)

    # Set column to 2 rows
    double_columns_num = pd.MultiIndex.from_arrays(
        [
            ["Model 1", " ", "Model 2", " ", "Model 3", " ", "Model 4", " "],
            ["ID", "Version", "ID", "Version", "ID", "Version", "ID", "Version"],
        ]
    )

    # Change column
    output_frame.columns = double_columns_num

    # setting backup
    backup_max_rows = pd.options.display.max_rows
    # output limit clear
    pd.set_option("display.max_rows", None)
    display(output_frame)
    # setting restore
    pd.set_option("display.max_rows", backup_max_rows)


print("Create access lib client...")

auth_param = {}
auth_param["console_endpoint"] = json_load["console_endpoint"]
auth_param["portal_authorization_endpoint"] = json_load["portal_authorization_endpoint"]
auth_param["client_secret"] = json_load["client_secret"]
auth_param["client_id"] = json_load["client_id"]

try:
    config_obj = Config(**auth_param)
except Exception as e:
    # EXCEPTION
    raise e

# Instantiate Console Access Library Client.
client_obj = Client(config_obj)

# Get device and model information for deploying to device
get_devices_and_ai_models(client_obj)

### Prepare configurations

#### Create Configuration file (only at first time)

In [ ]:
configuration_path = Path("./configuration.json")
if not configuration_path.is_file():
    model_sas_url = "YOUR_SAS_URL_OF_AI_MODEL"
    model_id = "YOUR_AI_MODEL_ID"
    # model_version_number = "1.00"

    ppl_file = f"{VNSAPP_DIR}/vision_app_zonedetection.wasm"
    app_name = "YOUR_APP_NAME"
    app_version_number = APP_VERSION_NUMBER

    config_id = "YOUR_CONFIG_ID"

    device_id = "YOUR_DEVICE_ID"

    command_parameter_file_name = "command_parameter_file.json"

    with open(configuration_path, "w") as fp:
        configuration = {
            "import_model": {
                "model_id": model_id,
                "model": model_sas_url,
                # "converted": false,
                "vendor_name": "YOUR_VENDOR_NAME",
                "comment": "YOUR_MODEL_COMMENT",
                # "network_type": "0",
                "labels": [""],
            },
            "import_app": {
                "ppl_file": ppl_file,
                "app_name": app_name,
                "version_number": app_version_number,
                "comment": "YOUR_APP_COMMENT",
            },
            "deploy_model": {
                "should_create_deploy_config": True,
                "config_id": config_id,
                "create_config": {
                    "comment": "YOUR_CONFIGURATION_COMMENT",
                    "model_id": model_id,
                    # "model_version_number": ""
                },
                "device_ids": [device_id],
                # "replace_model_id": "",
                "comment": "YOUR_MODEL_DEPLOY_COMMENT",
            },
            "deploy_app": {
                "app_name": app_name,
                "version_number": app_version_number,
                "device_ids": [device_id],
                "comment": "YOUR_APP_DEPLOY_COMMENT",
            },
            "command_parameter_file_name": command_parameter_file_name
        }
        json.dump(configuration, fp, indent=4)
    print("./configuration.json is created.")
else:
    print("./configuration.json already exists.")

#### Edit Configuration file

Edit the parameters in [configuration.json](./configuration.json).

|Setting|Description|Range|Required/Optional|Remarks
|:--|:--|:--|:--|:--|
|**`import_model`**||Object.<br>See following table of import_model|Required||
|**`import_app`**||Object.<br>See following table of import_app|Required||
|**`deploy_model`**||Object.<br>See following table of deploy_model|Required||
|**`deploy_app`**||Object.<br>See following table of deploy_app|Required||
|**`command_parameter_file_name`**|The filename of the Command Parameter you want to save.|String.|Required|Saving filename in the SDK.<br>The file is used for uploading to "**Console for AITRIOS**"|

##### import_model

|Setting|Description|Range|Required/Optional|Remarks
|:--|:--|:--|:--|:--|
|**`model_id`**|The ID of the AI model you want to import|String. <br>See NOTE. |Required|Used for "**Console Access Library**" API:<br>**`ai_model.ai_model.AIModel.import_base_model`**<br>**`ai_model.ai_model.AIModel.get_base_model_status`**<br>**`ai_model.ai_model.AIModel.publish_model`** |
|**`model`**|Path to SAS URI for AI model|SAS URI. <br>See NOTE. |Required|Used for "**Console Access Library**" API:<br>**`ai_model.ai_model.AIModel.import_base_model`**|
|**`vendor_name`**|vendor name|String. <br>See NOTE. |Optional|Used for "**Console Access Library**" API:<br>**`ai_model.ai_model.AIModel.import_base_model`**|
|**`comment`**|Description of the AI model and version|String. <br>See NOTE. |Optional|Used for "**Console Access Library**" API:<br>**`ai_model.ai_model.AIModel.import_base_model`**|
|**`labels`**|Label names|["label01", "label02", ...]<br>See NOTE. |Optional|Used for "**Console Access Library**" API:<br>**`ai_model.ai_model.AIModel.import_base_model`**|

> **NOTE**
>
> See [API Reference](https://developer.aitrios.sony-semicon.com/development-guides/reference/api-references/) of Console Access Library for other restrictions.

##### import_app

|Setting|Description|Range|Required/Optional|Remarks
|:--|:--|:--|:--|:--|
|**`ppl_file`**|The path to the "**Vision and Sensing Application**" file for importing |Absolute path or relative path from configuration.json/Notebook(*.ipynb)|Required||
|**`app_name`**|The name of the "**Vision and Sensing Application**" you want to import|String. <br>See NOTE. |Required|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.import_device_app`**|
|**`version_number`**|The version number of the "**Vision and Sensing Application**" you want to import |String. <br>See NOTE. |Required|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.import_device_app`** |
|**`comment`**|The description of the "**Vision and Sensing Application**" |String. <br>See NOTE. |Optional|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.import_device_app`** |

> **NOTE**
>
> See [API Reference](https://developer.aitrios.sony-semicon.com/development-guides/reference/api-references/) of Console Access Library for other restrictions.

##### deploy_model

|Setting||Description|Range|Required/Optional|Remarks
|:--|:--|:--|:--|:--|:--|
|**`should_create_deploy_config`**||Whether to register a new deploy configuration.<br>If true, create a new configuration; if false, use an already registered configuration specified by **`config_id`**.|true or false|Required||
|**`config_id`**||The ID of the deploy configuration you want to use for deployment.|String.<br>See NOTE.|Required|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.create_deploy_configuration`**<br>**`deployment.deployment.Deployment.deploy_by_configuration`**|
|**`create_config`**|**`comment`**|Description of the configuration.|String. <br>See NOTE.|Optional|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.create_deploy_configuration`**|
||**`model_id`**|The ID of the model you want to deploy.|String.<br>See NOTE.|Optional<br> Required if **`should_create_deploy_config`** is true.|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.create_deploy_configuration`**|
||**`model_version_number`**|The version of the model you want to deploy.|String.<br>See NOTE.|Optional|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.create_deploy_configuration`**|
|**`device_ids`**||List of device ids on which you want to deploy your model.|List of string.|Required|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.deploy_by_configuration`**|
|**`replace_model_id`**||The ID of the model you want to replace.|String.<br>See NOTE.|Optional|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.deploy_by_configuration`**|
|**`comment`**||Description of the deployment.|String.<br>See NOTE.|Optional|Used for "**Console Access Library**" API:<br>**`deployment.deployment.Deployment.deploy_by_configuration`**|

> **NOTE**
>
> See [API Reference](https://developer.aitrios.sony-semicon.com/development-guides/reference/api-references/) of Console Access Library for other restrictions.

##### deploy_app

|Setting|Description|Range|Required/Optional|Remarks
|:--|:--|:--|:--|:--|
|**`app_name`**|The name of the "**Vision and Sensing Application**" you want to deploy.|String.<br>See NOTE.|Required|Used for "**Console Access Library**" API: <br> **`deployment.deployment.Deployment.deploy_device_app`** <br> **`deployment.deployment.Deployment.get_device_app_deploys`**|
|**`version_number`**|The version number of the "**Vision and Sensing Application**" you want deploy.|String.<br>See NOTE.|Required|Used for "**Console Access Library**" API: <br> **`deployment.deployment.Deployment.deploy_device_app`** <br> **`deployment.deployment.Deployment.get_device_app_deploys`**|
|**`device_ids`**|List of device ids on which you want to deploy your "**Vision and Sensing Application**".|List of strings.|Required|Used for "**Console Access Library**" API: <br> **`deployment.deployment.Deployment.deploy_device_app`**|
|**`comment`**|Description of the deployment.|String.<br>See NOTE.|Optional|Used for "**Console Access Library**" API: <br> **`deployment.deployment.Deployment.deploy_device_app`**|

> **NOTE**
>
> See [API Reference](https://developer.aitrios.sony-semicon.com/development-guides/reference/api-references/) of Console Access Library for other restrictions.

### Load Configurations (for importing and deploying)

In [ ]:
configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file.
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file.
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration.
jsonschema.validate(json_load, json_schema)

IMPORT_MODEL_SCHEMA_PATH = (
    f"{SDK_ENV_ROOT_DIR}/tutorials/3_prepare_model/develop_on_sdk/"
    "3_import_to_console/configuration_schema.json"
)
IMPORT_APP_SCHEMA_PATH = (
    f"{SDK_ENV_ROOT_DIR}/tutorials/4_prepare_application/"
    "2_import_to_console/configuration_schema.json"
)
DEPLOY_MODEL_SCHEMA_PATH = (
    f"{SDK_ENV_ROOT_DIR}/tutorials/3_prepare_model/develop_on_sdk/"
    "4_deploy_to_device/deploy_to_device/configuration_schema.json"
)
DEPLOY_APP_SCHEMA_PATH = (
    f"{SDK_ENV_ROOT_DIR}/tutorials/4_prepare_application/"
    "3_deploy_to_device/configuration_schema.json"
)


def validate_objects(schema_path: str, key: str):
    configuration_schema_path = Path(schema_path)
    validate_symlink(configuration_schema_path)
    # Load configuration schema file.
    with open(configuration_schema_path, "r") as f:
        json_schema = json.load(f)
    # Validate configuration.
    jsonschema.validate(json_load[key], json_schema)


validate_objects(IMPORT_MODEL_SCHEMA_PATH, "import_model")
validate_objects(IMPORT_APP_SCHEMA_PATH, "import_app")
validate_objects(DEPLOY_MODEL_SCHEMA_PATH, "deploy_model")
validate_objects(DEPLOY_APP_SCHEMA_PATH, "deploy_app")

print("./configuration.json is loaded.")

### Import AI model to "**Console**"

#### Import AI model

In [ ]:
def import_model(json_load):
    """Import AI model to Console"""

    model_id = json_load["model_id"]
    model_import_param = {}
    model_import_param["model_id"] = model_id
    model_import_param["model"] = json_load["model"]
    model_import_param["network_type"] = "0"
    if "vendor_name" in json_load:
        model_import_param["vendor_name"] = json_load["vendor_name"]
    if "comment" in json_load:
        model_import_param["comment"] = json_load["comment"]
    if "labels" in json_load:
        model_import_param["labels"] = copy.deepcopy(json_load["labels"])

    # Call an API to import AI model into Console for AITRIOS
    try:
        ai_model_obj = client_obj.get_ai_model()
        # print(model_import_param)
        response = ai_model_obj.import_base_model(**model_import_param)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print("Start to import the AI model." + " \n\tmodel_id: " + model_id)


import_model(json_load["import_model"])

#### Convert AI model

In [ ]:
def convert_model(json_load):
    """Convert AI model on Console"""

    model_id = json_load["model_id"]
    # Call an API to convert AI model on Console for AITRIOS
    try:
        ai_model_obj = client_obj.get_ai_model()
        response = ai_model_obj.publish_model(model_id=model_id)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print("Start to convert the AI model." + " \n\tmodel_id: " + model_id)


convert_model(json_load["import_model"])

#### Check AI model status after conversion

To complete the conversion, ensure that the conversion status is **`Conversion completed`**.

After you start the conversion, run the following code cell to check the status.

> **NOTE**
>
> Conversion takes several minutes.

In [ ]:
def get_base_model_status(json_load):
    """Get AI model status on Console"""

    model_id = json_load["model_id"]
    # AI model status on Console
    status_dictionary = {
        "00": "Conversion project created",
        "01": "Importing completed (Before conversion)",
        "02": "Converting...",
        "03": "Conversion failed",
        "04": "Converted",
        "05": "Adding to configuration",
        "06": "Conversion failed",
        "07": "Conversion completed",
        "11": "Saving",
    }
    # Flag for check
    exist_flag = False
    # Call an API for get AI model info
    try:
        ai_model_obj = client_obj.get_ai_model()
        # print(model_id)
        response = ai_model_obj.get_base_model_status(model_id)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    # Create output list
    if "projects" in response:
        project = response["projects"][0]
        if "versions" in project:
            version_status = project["versions"][0]["version_status"]
            exist_flag = True

    if exist_flag:
        message = status_dictionary.get(
            version_status, "Unknown status '" + version_status + "'"
        )
        return (version_status, message)
    else:
        raise Exception("AI model is not found. (model_id: " + model_id + ")")


status, message = get_base_model_status(json_load["import_model"])
print(message)

### Import "**Vision and Sensing Application**" to "**Console**"

#### Import "**Vision and Sensing Application**"

In [ ]:
def import_app(json_load):
    """Import Vision and Sensing Application to Console"""

    # encode base 64
    def convert_file_to_b64_string(file_path):
        with open(file_path, "rb") as f:
            return base64.b64encode(f.read())

    ppl_file = Path(json_load["ppl_file"].replace(os.path.sep, "/"))
    validate_symlink(ppl_file)

    file_content = convert_file_to_b64_string(ppl_file)

    file_name = os.path.basename(ppl_file)

    print(json_load["ppl_file"] + " is loaded.")

    vasa_import_param = {}
    vasa_import_param["app_name"] = json_load["app_name"]
    vasa_import_param["version_number"] = json_load["version_number"]
    vasa_import_param["compiled_flg"] = "0"
    vasa_import_param["entry_point"] = "main"
    vasa_import_param["file_name"] = file_name
    vasa_import_param["file_content"] = file_content
    if "comment" in json_load:
        vasa_import_param["comment"] = json_load["comment"]

    # Call an API to import Vision and Sensing Application into Console for AITRIOS
    try:
        deployment_obj = client_obj.get_deployment()
        # print(vasa_import_param)
        response = deployment_obj.import_device_app(**vasa_import_param)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print(
        "Start to import the Vision and Sensing Application."
        + " \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
    )


import_app(json_load["import_app"])

#### Check "**Vision and Sensing Application**" status

To complete the conversion, ensure that the conversion status is **`Conversion completed`**.

After you start the conversion, run the following code cell to check the status.

> **Note**
>
> Import and conversion take several minutes.

In [ ]:
def get_device_app_status(json_load):
    """Get Vision and Sensing Application status on Console"""

    app_name = json_load["app_name"]
    version_number = json_load["version_number"]

    # Status of Vision and Sensing Application on Console
    status_dictionary = {
        "0": "Importing completed (Before conversion)",
        "1": "Converting...",
        "2": "Conversion completed",
        "3": "Conversion failed",
    }
    # Flag for import check
    import_flag = False
    # Call an API to get Vision and Sensing Application info
    try:
        deployment_obj = client_obj.get_deployment()
        response = deployment_obj.get_device_apps()
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    # Create output list
    apps = response.get("apps", [])
    for app in apps:
        if "name" in app and app["name"] == app_name:
            versions = app.get("versions", [])
            for version in versions:
                if "version" in version and version["version"] == version_number:
                    import_flag = True
                    version_status = version.get("status", "")
                    break
            if import_flag:
                break
    if import_flag:
        return status_dictionary.get(
            version_status, "Unknown status '" + version_status + "'"
        )
    else:
        raise Exception(
            "Vision and Sensing Application is not found. "
            + " \n\tapp_name: "
            + app_name
            + "\n\tversion_number: "
            + version_number
        )


def check_app_status(json_load):
    """Check Vision and Sensing Application status on Console"""

    get_status = get_device_app_status(json_load)
    print(
        get_status
        + " \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
    )


check_app_status(json_load["import_app"])

### Create deploy configuration in "**Console**"

To deploy AI model to Edge AI Device, deploy configuration includes information of AI model and Edge AI Device to deploy.

In [ ]:
def create_deploy_config(json_load):
    """Create deploy configuration on Console"""

    # Set values
    should_create_deploy_config = json_load["should_create_deploy_config"]

    if should_create_deploy_config is True:
        config_param = {}
        config_param["config_id"] = json_load["config_id"]
        if "create_config" in json_load:
            config_param["model_id"] = json_load["create_config"]["model_id"]
            if "model_version_number" in json_load["create_config"]:
                config_param["model_version_number"] = json_load["create_config"][
                    "model_version_number"
                ]
            if "comment" in json_load["create_config"]:
                config_param["comment"] = json_load["create_config"]["comment"]

    if should_create_deploy_config is True:
        # Call an API to create deploy configuration
        try:
            deployment_obj = client_obj.get_deployment()
            # print(config_param)
            response = deployment_obj.create_deploy_configuration(**config_param)
        except Exception as e:
            # EXCEPTION
            raise e

        # response error check
        if "result" in response and response["result"] != "SUCCESS":
            # ERROR
            raise ValueError("ERROR", response)

        # SUCCESS
        print("Deploy configuration was created.")


create_deploy_config(json_load["deploy_model"])

### Deploy model to device

#### Deploy model

In [ ]:
def deploy_model(json_load):
    """Deploy AI model to device"""

    device_ids_list = json_load["device_ids"]

    # Set values
    deploy_param = {}
    deploy_param["config_id"] = json_load["config_id"]
    deploy_param["device_ids"] = ",".join(device_ids_list)
    if "replace_model_id" in json_load:
        deploy_param["replace_model_id"] = json_load["replace_model_id"]
    if "comment" in json_load:
        deploy_param["comment"] = json_load["comment"]

    # Call an API to deploy the model to device
    try:
        deployment_obj = client_obj.get_deployment()
        # print(deploy_param)
        response = deployment_obj.deploy_by_configuration(**deploy_param)
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print("Start to deploy the model.")


deploy_model(json_load["deploy_model"])

#### Check deployment status

To complete the deployment, ensure that the deployment status is **`Success`**.

After you start the deployment, run the following code cell to check the status.

> **NOTE**
>
> Deploying to devices takes several minutes to complete.

In [ ]:
def check_deploy_status(json_load):
    """Get and check deploy status of AI model"""

    device_ids_list: list = json_load["device_ids"]
    deploy_config_id: str = json_load["config_id"]

    # Deploy status on Console
    deploy_status_dictionary = {
        "0": "Deploying",
        "1": "Success",
        "2": "Fail",
        "3": "Cancel",
    }
    # Model status on Console
    model_status_dictionary = {
        "0": "Waiting for execution",
        "1": "Deploying",
        "2": "Success",
        "3": "Fail",
    }

    deploy_ids = []
    config_ids = []
    deploy_statuses = []
    model_statuses = []
    update_dates = []
    device_id_table = []
    for device_id in device_ids_list:
        # Call an API to get deploy history
        try:
            deployment_obj = client_obj.get_deployment()
            response = deployment_obj.get_deploy_history(device_id)
        except Exception as e:
            # EXCEPTION
            raise e

        # response error check
        if "result" in response and response["result"] != "SUCCESS":
            # ERROR
            raise ValueError("ERROR", response)

        # Create an output table
        deploys = response.get("deploys", [])
        cnt = 0
        for deploy in deploys:
            model = deploy.get("model", {})
            model_target_flg = model.get("model_target_flg", "")
            config_id = deploy.get("config_id", "")
            if model_target_flg == "1" and deploy_config_id == config_id:
                # Set device id
                if cnt == 0:
                    device_id_table.append(device_id)
                else:
                    # Fill a cell with a NAN
                    device_id_table.append(np.NaN)
                # Set deploy ID
                deploy_id = deploy.get("id", "")
                deploy_ids.append(deploy_id)
                # Set config ID
                config_ids.append(config_id)
                # Set deploy status
                deploy_status = deploy.get("deploy_status", "")
                deploy_statuses.append(
                    deploy_status_dictionary.get(
                        deploy_status, "Unknown status '" + deploy_status + "'"
                    )
                )
                # Set model status
                model_status = model.get("model_status", "")
                model_statuses.append(
                    model_status_dictionary.get(
                        model_status, "Unknown status '" + model_status + "'"
                    )
                )
                # Set update date
                update_dates.append(deploy.get("upd_date", ""))

                cnt += 1
                # Display up to 5 deployment results
                if cnt == 5:
                    break

    if len(deploy_ids) == 0:
        raise Exception("There is no data in the deploy history list.")

    output_frame = pd.DataFrame(
        {
            "device_id": device_id_table,
            "deploy_id": deploy_ids,
            "config_id": config_ids,
            "deploy_status": deploy_statuses,
            "model_status": model_statuses,
            "update_date": update_dates,
        }
    )
    output_frame = output_frame.fillna("-")
    # setting backup
    backup_max_rows = pd.options.display.max_rows
    # output limit clear
    pd.set_option("display.max_rows", None)
    display(output_frame)
    # setting restore
    pd.set_option("display.max_rows", backup_max_rows)


check_deploy_status(json_load["deploy_model"])

### Deploy "**Vision and Sensing Application**" to device

#### Deploy "**Vision and Sensing Application**"

In [ ]:
def deploy_app(json_load):
    """Deploy Vision and Sensing Application to device"""

    param_dict = json_load.copy()

    device_ids = param_dict["device_ids"]
    device_ids_join = ",".join(device_ids)
    param_dict["device_ids"] = device_ids_join

    # Call an API to deploy Vision and Sensing Application from Console for AITRIOS to Edge AI
    # Device.
    try:
        deployment_obj = client_obj.get_deployment()
        # print(param_dict)
        response = deployment_obj.deploy_device_app(**param_dict)
    except Exception as e:
        # EXCEPTION
        raise e

    # Response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS
    print(
        "Start to deploy application. \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
        + "\n\tdevice_ids: "
        + device_ids_join
    )


deploy_app(json_load["deploy_app"])

#### Check deployment status

To complete the deployment, ensure that the deployment status is **`Success`**.

After you start the deployment, run the following code cell to check the status.

> **NOTE**
>
> Deploying to devices takes several minutes.

In [ ]:
def check_app_deploy_status(json_load):
    """Check deploy status of Vision and Sensing Application"""

    device_ids = json_load["device_ids"]

    status_dictionary = {"0": "Deploying", "1": "Success", "2": "Fail", "3": "Cancel"}

    response_statuses = []
    response_device_ids = []

    # Call an API to get Vision and Sensing Application info.
    try:
        deployment_obj = client_obj.get_deployment()
        response = deployment_obj.get_device_app_deploys(
            json_load["app_name"], json_load["version_number"]
        )
    except Exception as e:
        # EXCEPTION
        raise e

    # response error check
    if "result" in response and response["result"] != "SUCCESS":
        # ERROR
        raise ValueError("ERROR", response)

    # SUCCESS

    deploys = response.get("deploys", [])
    # Display deployment status of specified devices
    for deploy in deploys:
        if "devices" in deploy:
            devices = deploy.get("devices", [])
            for device in devices:
                if (
                    "device_id" in device
                    and device["device_id"] in device_ids
                    and device["latest_deployment_flg"] == "1"
                ):
                    response_device_ids.append(device.get("device_id", ""))
                    deploy_status = device.get("status", "")
                    response_statuses.append(
                        status_dictionary.get(
                            deploy_status, "Unknown status '" + deploy_status + "'"
                        )
                    )

    if len(response_device_ids) == 0:
        raise Exception(
            "Failed to get deploy status. Deploy history not found. \n\tapp_name: "
            + json_load["app_name"]
            + "\n\tversion_number: "
            + json_load["version_number"]
            + "\n\tdevice_ids: "
            + ",".join(device_ids)
        )

    print(
        "Deployment status of: \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
    )

    output_frame = pd.DataFrame(
        {"device_id": response_device_ids, "status": response_statuses}
    )

    # setting backup
    backup_max_rows = pd.options.display.max_rows
    # output limit clear
    pd.set_option("display.max_rows", None)
    # View results
    display(output_frame)
    # setting restore
    pd.set_option("display.max_rows", backup_max_rows)


check_app_deploy_status(json_load["deploy_app"])

## Evaluate

### Evaluate without Web App
Please see [Console User Manual](https://developer.aitrios.sony-semicon.com/en/documents/console-user-manual) for details.

#### Prepare Command Parameter File

Before running following cell, edit following **`model_id`**, **`ppl_parameter`** and **`command_parameter`** if needed.

In [ ]:
model_id = json_load["import_model"]["model_id"]

ppl_parameter = {
    "header": {
        "id": "00",
        "version": APP_VERSION_NUMBER
    },
    "dnn_output_detections": DNN_OUTPUT_DETECTIONS,
    "max_detections": MAX_DETECTIONS,
    "mode": 0,
    "zone": {
        "top_left_x": 10,
        "top_left_y": 168,
        "bottom_right_x": 198,
        "bottom_right_y": 258
    },
    "threshold": {
        "iou": 0.5,
        "score": 0.3
    },
    "input_width": INPUT_SIZE,
    "input_height": INPUT_SIZE
}

command_parameter = {
    "commands": [
        {
            "command_name": "StartUploadInferenceData",
            "parameters": {
                "Mode": 1,  # 0: image only, 1: image and inference, 2: inference only, default: 0
                "NumberOfImages": 1,  # default: 0, 0 means uploading images until stop
                "PPLParameter": ppl_parameter,
                "ModelId": model_id,
                # "UploadInterval": 30, # 30-2592000 default: 30 (a unit is 1/30 seconds)
                # "UploadMethod": "BlobStorage", # default: "BlobStorage"
                # "FileFormat": "JPG", # "JPG", "BMP" default: "JPG"
                # "UploadMethodIR": "Mqtt", # default: "Mqtt"
                # "NumberOfInferencesPerMessage": 1, # 1-100 default: 1
                # "MaxDetectionsPerFrame": 5, # 1-5 default: 5
                # "CropHOffset": 0, # 0-4055 default: 0
                # "CropVOffset": 0, # 0-3039 default: 0
                # "CropHSize": 4056, # 0-4056 default: 4056
                # "CropVSize": 3040, # 0-3040 default: 3040
            }
        }
    ]
}

command_parameter_file_name = json_load["command_parameter_file_name"]

with open(command_parameter_file_name, "w") as fp:
    json.dump(command_parameter, fp, indent=4)
    print(
        f"Command Parameter File of StartUploadInferenceData \
            is saved at ./{command_parameter_file_name}"
    )

#### Upload Command Parameter File of StartUploadInferenceData including PPL Parameter

Upload the saved Command Parameter File from "**Settings**" in "**Console for AITRIOS**" Web UI.

#### Bind the Command Parameter File to Edge AI Device

Bind the Command Parameter File  from "**Manage device**" in "**Console for AITRIOS**" Web UI.

#### Execute StartUploadInferenceData

Start inference from "**Manage device**" in "**Console for AITRIOS**" Web UI.

> **NOTE**
>
> If there is no subject of images, the correct inference results will not be obtained.

#### Execute StopUploadInferenceData

Stop inference from "**Manage device**" in "**Console for AITRIOS**" Web UI.

#### Check image and metadata of inference result

Check images and inference results from "**Check data**" in "**Console for AITRIOS**" Web UI.

### Evaluate with Web App

You can run a web application of ["**Zone Detection**"](https://developer.aitrios.sony-semicon.com/development-guides/tutorials/sample-application/) sample application outside the SDK environment.

If you use the web application, you can use the Wasm file of "**Vision and Sensing Application**" created with this notebook.

> **Note**
> 
> The web application is made for using dataset annotated in "**Console for AITRIOS**" and AI model trained in "**Console for AITRIOS**".
>
> So following PPL Parameter is used in the web application.
> 
> - **`input_size`** is 320
> - **`dnn_output_detections`** is 64
>
> The dataset annotated in "**Console for AITRIOS**" can't be exported to the SDK environment.
>
> The AI model trained in "**Console for AITRIOS**" can't be exported to the SDK environment.